<a href="https://colab.research.google.com/github/kiitaamuuraa/Asobiba/blob/main/MyFirstConditionalTextGenerationWithTransformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 河原さんをほめる  


## 1. 一番簡単なボット

In [ ]:
import random

In [ ]:
techniques = [
              'キック','ローキック','ハイキック','二段蹴り','前蹴り', # 蹴り
              'ジャブ','ストレート','ワンツー','フック', 'アッパー', # パンチ
              'スウェイ', 'ダッキング', 'クリンチ', # 防御
              '首相撲'
              ]

In [ ]:
technique = techniques[random.randint(0, len(techniques)-1)]

sent = [
        "河原さんの{}はやっぱ流石ですね。".format(technique),
        "河原さんの{}はキレが違いますね。".format(technique),
        "さっきの河原さんの{}が早すぎて見えませんでした。".format(technique),
        "河原さんの{}には素人じゃ対応できないですね。".format(technique),
        ]

print(sent[random.randint(0, len(sent)-1)])

河原さんのジャブには素人じゃ対応できないですね。


## 2. 上記の文章の続きを事前学習済みの生成モデルを用いて生成

In [ ]:
!pip install transformers
!pip install sentencepiece # T5のトークナイザに必要

In [ ]:
from transformers import T5Tokenizer, AutoModelForCausalLM, T5ForConditionalGeneration

In [ ]:
model_name = 'GPT2' # とりあえずGPT2を使ってみる

if model_name == 'GPT2':
    tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
    model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")
elif model_name == 'T5':
    tokenizer = T5Tokenizer.from_pretrained("sonoisa/t5-base-japanese")
    model = T5ForConditionalGeneration.from_pretrained("sonoisa/t5-base-japanese")

model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(32000, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2):

In [ ]:
tokenized = tokenizer(sent, padding=True, truncation=True, max_length=512, return_tensors="pt" )
tokenized.to('cuda')        

{'input_ids': tensor([[    9,  9813,   774,    10,   743,   352,    11,    26, 10010,   490,
           284,  2767,   969,     8,     2,     3,     3,     3],
        [    9,  9813,   774,    10,   743,   352,    11,   260,   215,    12,
          4251,  3418,   969,     8,     2,     3,     3,     3],
        [    9,   338, 22265,    10,  9813,   774,    10,   743,   352,    12,
          1745, 21858, 11096,  9621,    19,    29,     8,     2],
        [    9,  9813,   774,    10,   743,   352,    42, 24847,  8162,  1270,
           997,  2767,   969,     8,     2,     3,     3,     3]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]],
       device='cuda:0')}

In [1]:
# TODO: ハイパラ真面目に検討
# https://huggingface.co/blog/how-to-generate

max_length = random.randint(50, 200)
print(max_length)
outputs = model.generate(tokenized['input_ids'],
                        max_length=max_length,
                        temperature=1.5,
                        num_beams=8,  
                        diversity_penalty=1.0,  
                        num_beam_groups=4,  
                        num_return_sequences=2,  
                        repetition_penalty=8.0, 
                        early_stopping=True,
                        no_repeat_ngram_size=2, 
                        )

for output in outputs:
    print(tokenizer.decode(output, skip_special_tokens=True))

NameError: ignored